In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/Disaster_Tweets

In [ ]:
!pip install -r requirements.txt


# FEATURE ENGINEERING AND MODELING

Quá trình tạo đặc trưng cho mỗi dòng dữ liệu text.

Lựa chọn và huấn luyện mô hình để giải quyết bài toán.

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
%cd /content/drive/My Drive/Disaster_Tweets/src

Đọc dữ liệu đã được tiền xử lý: `light_pre_train.csv` và `light_pre_test.csv` vào 2 dataframe: `train_df` và `test_df`

In [ ]:
train_df = pd.read_csv('./preprocessing/light_pre_train.csv')
test_df = pd.read_csv('./preprocessing/light_pre_test.csv')

## 1. Chuẩn bị dữ liệu

### 1.1 Tổng quan:
- Chia dữ liệu trên `train_df` theo tỉ lệ `90:10` tương ứng để fine tune trên mô hình BERT với 70% dữ liệu và đánh giá mô hình trên 10% dữ liệu còn lại.

- Sao chép 100% dữ liệu trên `train_df` sang `train_df_shuffle`, sau đó xáo trộn 100% dữ liệu đó, và tiếp tục fine tune với mô hình BERT đã được fine tuning trước đó, trước khi nộp bài đánh giá trên kaggle.

- Dùng mô hình pretrained `bert-base-uncased` (tiếng anh) và module BertTokenizer từ thư viện `transformer` để tokenize văn bản(text) thành vector tương ứng. Thực hiện trên cả train_df, test_df và train_df_shuffle.

- Dựa vào số lượng token (từ) được tách ra (tokenize) từ các câu trên train_df, để lựa chọn `max_length` - số chiều của vector embedding khi fine tune bằng mô hình BERT.


### 1.2 Sao chép và xáo trộn dữ liệu

Sao chép dữ liệu `train_df` sang `train_df_shuffle` và xáo xộn nó.

In [ ]:
train_df_shuffle = train_df.sample(frac=1, random_state=12)

In [ ]:
train_df_shuffle

Lấy text và target trong `train_df_shuffle`

In [ ]:
all_train_labels = train_df_shuffle.target
all_train_text = train_df_shuffle.text

Lấy ra cột `text`, `target` trên `train_df` và cột `text` trên `test_df`

In [ ]:
train_text = train_df.text
train_labels = train_df.target
test_text = test_df.text

### 1.3 Tokenine văn bản và lựa chọn siêu tham số max_length

Cài đặt thư viện transformer

In [ ]:
!pip install transformers

`BertTokenizer`: dùng để token văn bản thành vector


In [ ]:
from transformers import BertTokenizer,TFBertForSequenceClassification

Dùng pretrained `bert-base-uncased` để tokenize văn bản thành vector.

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Tokenize trên `train_text`

In [ ]:
token_lens = []
for txt in list(train_text):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))

In [ ]:
tokenizer.encode("Hoang Duc Nhat", max_length=512, truncation=True)

Ở đây max_length = 512 sẽ giới hạn là độ dài của mỗi câu. Nếu câu nào có độ dài lớn hơn 512, thì sẽ cắt bỏ (truncation = True). Tuy nhiên, toàn bộ các câu trong bộ corpus không tồn tại câu nào có độ dài lớn như vậy. Ta thử xem phân phối của chúng trên tập train.

In [ ]:
sns.displot(token_lens)
plt.xlim([0, 100])
plt.xlabel('Số lượng token', fontweight = "bold")
plt.ylabel('Tần suất', fontweight = "bold")
plt.title("Mối quan hệ giữa độ dài của các câu theo từ và tần suất của nó", fontweight = "bold")
plt.show()

Dựa vào đồ thị, ta sẽ chọn siêu tham số `max_lengh` = 45 hoặc 50 hoặc 64 để train mô hình.

## 2. Feature Engineering

Chuyển text thành vector tương ứng trong bộ vocabulary lấy từ pretrained bert-base-uncased

In [ ]:
MAX_SEQ_LEN = 50

In [ ]:
def bert_tokenizer(text):
    encoding = tokenizer.encode_plus(
    text,
    max_length=MAX_SEQ_LEN,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',  # Return PyTorch tensors
    )
    return encoding['input_ids'][0], encoding['attention_mask'][0]

In [ ]:
print("Lấy ví dụ, tạo đặc trưng cho chuỗi: Hoang Duc Nhat Minh")
bert_tokenizer("Hoang Duc Nhat Minh")

Chuyển text thành vector trên tập `train_text`, và lưu vào `train_tokenized_list`

In [ ]:
train_tokenized_list = []
train_attn_mask_list = []
for text in list(train_text):
    tokenized_text, attn_mask = bert_tokenizer(text)
    train_tokenized_list.append(tokenized_text.numpy())
    # train_attn_mask_list.append(attn_mask.numpy())

Chuyển text thành vector trên tập `test_text`, và lưu vào `test_tokenized_list`.

In [ ]:
# test data tokenization for testing
test_tokenized_list = []
test_attn_mask_list = []
for text in list(test_text):
    tokenized_text, attn_mask = bert_tokenizer(text)
    test_tokenized_list.append(tokenized_text.numpy())
    # test_attn_mask_list.append(attn_mask.numpy())

Chuyển text thành vector trên tập `all_train_text`, và lưu vào `test_tokenised_text_df`

In [ ]:
all_train_tokenized_list = []
for text in list(all_train_text):
    tokenized_text, attn_mask = bert_tokenizer(text)
    all_train_tokenized_list.append(tokenized_text.numpy())

Chuyển `train_tokenized_list`, `all_train_tokenized_list` và `test_tokenized_list` sang dạng dataframe

In [ ]:
train_tokenised_text_df = pd.DataFrame(train_tokenized_list)
all_train_tokenised_text_df = pd.DataFrame(all_train_tokenized_list)
test_tokenised_text_df = pd.DataFrame(test_tokenized_list)

Từ `train_tokenized_text_df` chia thành 90% `X_train` và 10% `X_val`

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_tokenised_text_df, train_labels, test_size=0.1, random_state=4)

In [ ]:
print(f'X_train input shape {X_train.shape}, train label shape {y_train.shape}')
print(f'X_val input shape {X_val.shape}, validation label shape {y_val.shape}')

## 2. Fine tune trên mô hình BERT

### 2.1 Tổng quan
Giai đoạn 1: Fine tune với mô hình BERT trên `X_train` (90% dữ liệu) và đánh giá trên `X_val` (10%) với 2 epoch.

Giai đoạn 2: Tiếp tục fine tune trên `all_train_text` với cấu hình: learning rate 5e-6, 1 epoch.

Các siêu tham số (hyperparameter) được dùng để tuning:
- `Learning rate` ở cả 2 giai đoạn.
- `max_length`: số lượng chiều của vector feature embedding, với các giá trị 45, 50, 55, 64.


### 2.2 Xây dựng hàm đánh giá mô hình

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
# Đánh giá mô hình
def eval_model(predictions):
    print(accuracy_score(y_val, predictions))
    # Compute fpr, tpr, thresholds and roc auc
    fpr, tpr, thresholds = roc_curve(np.array(y_val), np.array(predictions))
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()
    print(classification_report(y_val, np.array(predictions), target_names=["not disaster", "disaster"]))

## 3.3 Chi tiết

#### 3.3.1 Giai đoạn 1: 

Fine tune với mô hình BERT trên `X_train` (90% dữ liệu) và đánh giá trên `X_val` (10%)

Lấy số lượng class của dữ liệu

In [ ]:
num_classes = len(train_labels.unique())
num_classes

Gọi mô hình pretrained `TFBertForSequenceClassification`


In [ ]:
from transformers import TFBertForSequenceClassification

In [ ]:
bertClassifier = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)
print('\nBert Model', bertClassifier.summary())

Cấu hình hàm loss, optimizer, metric
- Hàm loss: cross entropy
- Optimizer: Adam
- Callback ModelCheckpoint: lưu mô hình
- Learning rate: 1e-5

In [ ]:
checkpoint_path = "./models/my_bert.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 verbose=1)



loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bertClassifier.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Fine tune mô hình với batch_size = 32 và 2 epoch

In [ ]:
bertHistory = bertClassifier.fit(X_train,
                       y_train,
                       batch_size=32,
                       epochs=3,
                       validation_data=(X_val, y_val),
                       callbacks=[model_callback])

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(bertHistory.history['accuracy'])
plt.plot(bertHistory.history['val_accuracy'])
plt.title('Độ chính xác (accuracy) mô hình')
plt.ylabel('Độ chính xác (accuracy)', fontweight = "bold")
plt.xlabel('Epoch', fontweight = "bold")
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(bertHistory.history['loss'])
plt.plot(bertHistory.history['val_loss'])
plt.title('Độ mất mát (loss) mô hình', fontweight = 'bold')
plt.ylabel('Loss', fontweight = "bold")
plt.xlabel('epoch', fontweight = "bold")
plt.legend(['train', 'val'], loc='upper right')
plt.show()

#### 3.3.2 Giai đoạn 2: 

Tiếp tục fine tune trên `all_train_text` với cấu hình: learning rate = 5e-6, 1 epoch

In [ ]:
bertClassifier.load_weights(checkpoint_path)

Cấu hình lại hàm optimizer với learning rate nhỏ hơn, và learning rate bằng 5e-6

In [ ]:
optimizer2 = tf.keras.optimizers.Adam(learning_rate = 5e-7,epsilon=1e-08)

bertClassifier.compile(loss=loss,optimizer=optimizer2,metrics=[metric])

Fine tune mô hình với 1 epoch

In [ ]:
history_all = bertClassifier.fit(all_train_tokenised_text_df,
                       all_train_labels,
                       batch_size=32,
                       epochs=1)

In [ ]:
# predictions
test_pred_all = bertClassifier.predict(X_val)
tensor_test_predictions_all = tf.math.softmax(test_pred_all.logits, axis=1)
print(tensor_test_predictions_all)

# then use argmax after softmax to turn into labels
test_predictions_all = [list(bertClassifier.config.id2label.keys())[i] for i in tf.math.argmax(tensor_test_predictions_all, axis=1).numpy()]

In [ ]:
eval_model(test_predictions_all)

## 3. Dự đoán mô hình trên tập test và nộp bài lên Kaggle

In [ ]:
submitted_prediction = bertClassifier.predict(test_tokenised_text_df)
print(submitted_prediction)

In [ ]:
submitted_prediction = tf.math.softmax(submitted_prediction.logits, axis=1)
print(submitted_prediction)

Dùng hàm softmax để suy ra nhãn của từng mẫu dữ liệu

In [ ]:
submitted_prediction = [list(bertClassifier.config.id2label.keys())[i] for i in tf.math.argmax(submitted_prediction, axis=1).numpy()]

In [ ]:
print("Kết quả dự đoán")
print(submitted_prediction)

In [ ]:
df_sample = pd.read_csv('./dataset/sample_submission.csv')
df_sample['target'] = submitted_prediction
df_sample.to_csv('submission.csv', index = False)